In [46]:
!pip install polygeohasher
!pip install folium
!pip install uszipcode
%pip install pygeohash
!pip install geopandas

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from IPython.display import Image
import folium
from folium import IFrame
from folium.plugins import MarkerCluster
from folium import plugins
from datetime import datetime
import datetime as dt
import json
from scipy import stats
import pygeohash as gh
import geopandas as gpd

import os

In [49]:
sampling_fraction = 0.8
geohash_precision = 6

In [50]:
air =  pd.read_csv('/content/drive/MyDrive/foundation_of_DS/project/Data/NYC_air_quality_1.csv')
air.head()

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin17,bin18,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm25
0,NYCP2_CS01A,1631277304,40.847672,-73.869316,11,1,1,0,0,0,...,0,0,0,0,0,0,0,23.7,57.3,4.508813
1,NYCP2_CS01A,1631277308,40.847668,-73.869316,22,4,1,0,0,2,...,0,0,0,0,0,0,0,23.7,57.8,5.462420
2,NYCP2_CS01A,1631277313,40.847649,-73.869362,40,1,1,0,0,1,...,0,0,0,0,0,0,0,23.7,57.8,5.154881
3,NYCP2_CS01A,1631277318,40.847649,-73.869362,26,1,0,0,0,0,...,0,0,0,0,0,0,0,23.6,57.6,4.508813
4,NYCP2_CS01A,1631277323,40.847649,-73.869362,44,4,0,1,0,0,...,0,0,0,0,0,0,0,23.6,57.5,5.539503


In [51]:
air = \
air[(air['latitude'] != 0 ) & \
(air['longitude']!=0 )]

In [52]:
%%time
air['geohash']=air.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=geohash_precision), axis=1)

CPU times: user 7.64 s, sys: 132 ms, total: 7.77 s
Wall time: 10.4 s


In [53]:
# convert to Geopandas Geodataframe
%%time
#join with geojson polygons to calculate RMSE
gdf_air = gpd.GeoDataFrame(air,   geometry=gpd.points_from_xy(air.longitude, air.latitude))

CPU times: user 480 ms, sys: 17 ms, total: 497 ms
Wall time: 560 ms


In [54]:
gdf_air.shape

(169999, 33)

In [55]:
#simple random sampling (SRS)
#gdf_air=gdf_air.sample(frac=sampling_fraction)

In [56]:
gdf_air.shape

(169999, 33)

In [57]:
hoods = gpd.read_file("/content/drive/MyDrive/foundation_of_DS/project/Data/nyc_polygon.geojson")
hoods.head()

,neighborhood,boroughCode,borough,@id,geometry
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.84860 40.87167, -73.84582 40.870..."
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.74333 40.73888, -73.74371 40.739..."
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-74.16983 40.56108, -74.16982 40.561..."
3,Arlington,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-74.15975 40.64142, -74.15998 40.641..."
4,Arrochar,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-74.06078 40.59319, -74.06079 40.593..."


In [58]:
hoods['geometry'] = hoods.geometry.to_crs("epsg:3857")
hoods

,neighborhood,boroughCode,borough,@id,geometry
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8220788.214 4993431.406, -8220479.3..."
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8209070.244 4973902.938, -8209112.6..."
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8256547.374 4947814.753, -8256546.8..."
3,Arlington,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8255425.402 4959593.695, -8255451.0..."
4,Arrochar,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8244408.305 4952520.907, -8244409.6..."
...,...,...,...,...,...
305,Windsor Terrace,3,Brooklyn,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8235422.696 4962430.807, -8235402.3..."
306,Woodhaven,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8222316.990 4967599.072, -8221672.8..."
307,Woodlawn,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8221998.344 4997679.030, -8221975.2..."
308,Woodrow,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8257377.292 4944818.936, -8257371.5..."


In [59]:
gdf_air = gdf_air.set_crs('epsg:4326')
gdf_air['geometry'] = gdf_air.geometry.to_crs("epsg:3857")
gdf_air

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm25,geohash,geometry
0,NYCP2_CS01A,1631277304,40.847672,-73.869316,11,1,1,0,0,0,...,0,0,0,0,0,23.7,57.3,4.508813,dr72rh,POINT (-8223094.642 4989899.216)
1,NYCP2_CS01A,1631277308,40.847668,-73.869316,22,4,1,0,0,2,...,0,0,0,0,0,23.7,57.8,5.462420,dr72rh,POINT (-8223094.642 4989898.627)
2,NYCP2_CS01A,1631277313,40.847649,-73.869362,40,1,1,0,0,1,...,0,0,0,0,0,23.7,57.8,5.154881,dr72rh,POINT (-8223099.763 4989895.831)
3,NYCP2_CS01A,1631277318,40.847649,-73.869362,26,1,0,0,0,0,...,0,0,0,0,0,23.6,57.6,4.508813,dr72rh,POINT (-8223099.763 4989895.831)
4,NYCP2_CS01A,1631277323,40.847649,-73.869362,44,4,0,1,0,0,...,0,0,0,0,0,23.6,57.5,5.539503,dr72rh,POINT (-8223099.763 4989895.831)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169994,NYCP2_CS03A,1631457109,40.823353,-73.890488,115,11,2,0,1,0,...,0,0,0,0,0,24.6,54.8,5.460360,dr72nx,POINT (-8225451.499 4986321.083)
169995,NYCP2_CS03A,1631457114,40.823349,-73.890480,132,8,2,0,0,0,...,0,0,0,0,0,24.6,54.8,5.298209,dr72nx,POINT (-8225450.608 4986320.495)
169996,NYCP2_CS03A,1631457119,40.823349,-73.890480,147,14,0,0,0,0,...,0,0,0,0,0,24.6,54.8,6.470661,dr72nx,POINT (-8225450.608 4986320.495)
169997,NYCP2_CS03A,1631457124,40.823345,-73.890488,121,8,2,0,1,1,...,0,0,0,0,0,24.6,54.6,6.424142,dr72nx,POINT (-8225451.499 4986319.907)


In [60]:
%%time
#BASELINE
# we join by sjoin, but we have geohash so, we sample stratified by geohash
# so, we join only to get the metrics but the stratified sampling is based on the fine-grained division (geohash in this case)
sjoined_air_original = gpd.sjoin(gdf_air, hoods, predicate="within")
sjoined_air_original.head(2)

CPU times: user 787 ms, sys: 357 ms, total: 1.14 s
Wall time: 2.78 s


,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,temperature,humidity,pm25,geohash,geometry,index_right,neighborhood,boroughCode,borough,@id
0,NYCP2_CS01A,1631277304,40.847672,-73.869316,11,1,1,0,0,0,...,23.7,57.3,4.508813,dr72rh,POINT (-8223094.642 4989899.216),38,Bronx Park,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
1,NYCP2_CS01A,1631277308,40.847668,-73.869316,22,4,1,0,0,2,...,23.7,57.8,5.462420,dr72rh,POINT (-8223094.642 4989898.627),38,Bronx Park,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...


In [61]:
n = len(pd.unique(sjoined_air_original['neighborhood']))

print("No.of.unique name values :", n)

No.of.unique name values : 48


In [62]:
sjoined_air_original.shape

(169995, 38)

In [63]:
%%time
# this is the NEW method that we want to compare with (in your case sample by H3 or Google's S2)
# sampling by geohash change fraction above (sampling rate) (20%, 40%, 60%, 80%)
sampled_geohash_data= sjoined_air_original.groupby('geohash').apply(lambda x: x.sample(frac=sampling_fraction))

CPU times: user 1.04 s, sys: 33.8 ms, total: 1.07 s
Wall time: 4.12 s


In [64]:
sampled_geohash_data.shape

(135993, 38)

In [65]:
!pip install h3

In [66]:
import h3

In [67]:
air.shape

(169999, 32)

In [68]:
H3_res = 10

def geo_2_h3(row):
  return h3.geo_to_h3(lat=float(row.latitude),lng=float(row.longitude),resolution = H3_res)

In [69]:
%%time
sjoined_air_original['h3_cell'] = sjoined_air_original.apply(geo_2_h3,axis=1)

CPU times: user 4.58 s, sys: 209 ms, total: 4.79 s
Wall time: 6.75 s


In [70]:
air_sample_pcount = (sjoined_air_original
                          .groupby('h3_cell')
                          .pm25
                          .agg(list)
                          .to_frame("ids")
                          .reset_index())
# No. points inside the hexagon
air_sample_pcount['count'] =(air_sample_pcount['ids']
                      .apply(lambda pm25_ids:len(pm25_ids)))
air_sample_pcount.shape

(1305, 3)

In [71]:
air_sample_pcount.sort_values('count',ascending=False)

,h3_cell,ids,count
96,8a2a100132e7fff,"[4.41621615, 5.344415292, 5.050625958, 6.04499...",12161
82,8a2a1001321ffff,"[11.94512709, 11.94512709, 12.45189569, 11.544...",10581
481,8a2a100a9157fff,"[4.460891362, 5.802586191, 5.802586191, 5.8025...",9308
80,8a2a1001320ffff,"[4.508813264, 5.462420278, 5.154880855, 4.5088...",6815
576,8a2a100a96b7fff,"[2.953916733, 3.552629716, 3.454585266, 2.2109...",5367
...,...,...,...
391,8a2a1001e417fff,[2.534738273],1
145,8a2a10015467fff,[3.429517792],1
65,8a2a10013067fff,[2.531530692],1
1231,8a2a100f6977fff,[2.24837233],1


In [72]:
air_sample_pcount

,h3_cell,ids,count
0,8a2a10002487fff,[3.571062523],1
1,8a2a1000248ffff,[3.571062523],1
2,8a2a10002497fff,[3.447159572],1
3,8a2a100024d7fff,"[3.447159572, 3.571062523]",2
4,8a2a100024dffff,[3.447159572],1
...,...,...,...
1300,8a2a1018b64ffff,"[5.041476755, 5.555489371, 3.931755641, 4.9705...",29
1301,8a2a1018b657fff,"[2.46271652, 5.470547648, 2.425009734, 6.29695...",32
1302,8a2a1018b65ffff,"[4.871851761, 2.46271652, 2.46271652, 5.277645...",44
1303,8a2a1018b66ffff,"[3.560544837, 5.367626584, 5.330154885, 5.3397...",22


In [73]:
air_sample_ncount = (sjoined_air_original
                          .groupby('h3_cell')
                          .neighborhood
                          .agg(list)
                          .to_frame("ids")
                          .reset_index())
# No. points inside the hexagon
air_sample_ncount['count'] =(air_sample_ncount['ids']
                      .apply(lambda neighborhood_ids:len(neighborhood_ids)))
air_sample_ncount.shape

(1305, 3)

In [74]:
air_sample_ncount

,h3_cell,ids,count
0,8a2a10002487fff,[Co-op City],1
1,8a2a1000248ffff,[Pelham Gardens],1
2,8a2a10002497fff,[Pelham Gardens],1
3,8a2a100024d7fff,"[Pelham Bay Park, Pelham Bay Park]",2
4,8a2a100024dffff,[Pelham Bay Park],1
...,...,...,...
1300,8a2a1018b64ffff,"[Van Cortlandt Park, Van Cortlandt Park, Van C...",29
1301,8a2a1018b657fff,"[Van Cortlandt Park, Van Cortlandt Park, Van C...",32
1302,8a2a1018b65ffff,"[Van Cortlandt Park, Van Cortlandt Park, Van C...",44
1303,8a2a1018b66ffff,"[Van Cortlandt Park, Van Cortlandt Park, Van C...",22


In [75]:
%%time
#BASELINE: original ==> this is the baseline we compare with, all data where sampling fraction =100, resembles  `without sampling`
geohash_data_original = sjoined_air_original.groupby('geohash').apply(lambda x: x.sample(frac=1))

CPU times: user 1.1 s, sys: 5.81 ms, total: 1.11 s
Wall time: 3.69 s


In [76]:
geohash_data_original.shape

(169995, 39)

Not really applicable to the H3 Uber, as it summarizes datapoint based on location rather than sample them.

In [77]:
# sampled data scenario #2
sampled_geohash_data2= sampled_geohash_data['neighborhood'].value_counts()
sampled_geohash_data2 = sampled_geohash_data2.reset_index()
sampled_geohash_data2.columns = ['neighborhood','count']
sampled_geohash_data2['neighborhood'] = sampled_geohash_data2['neighborhood'].astype(str)

In [78]:
# original data
geohash_data_original2= geohash_data_original['neighborhood'].value_counts()
geohash_data_original2 = geohash_data_original2.reset_index()
geohash_data_original2.columns = ['neighborhood','count']
geohash_data_original2['neighborhood'] = geohash_data_original2['neighborhood'].astype(str)

In [79]:
sampled_geohash_data2.head(6)

,neighborhood,count
0,Longwood,34019
1,Bronx Park,31616
2,Mott Haven,31449
3,Hunts Point,15407
4,Van Nest,5963
5,Port Morris,2509


In [80]:
geohash_data_original2.head(6)

,neighborhood,count
0,Longwood,42523
1,Bronx Park,39495
2,Mott Haven,39287
3,Hunts Point,19280
4,Van Nest,7477
5,Port Morris,3127


In [82]:
#reference geo-map
geohash_data_original2["neighborhood"].astype(str)
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(geohash_data_original2['count'].max() * (i/100.0)))

map_geohash_data_original2 = folium.Map(location=[40.730610, -73.935242], zoom_start=10)
map_geohash_data_original2.choropleth(geo_data=hoods, data=geohash_data_original2, \
                data_out = 'nyc_zip_test.json',
             columns=['neighborhood', 'count'],
             #threshold_scale= heatmap_scale,
             key_on='feature.properties.neighborhood',
             fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.9,nan_fill_color='white',
             legend_name='Number of Pickups')

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [83]:
map_geohash_data_original2

In [85]:
#NEW sampling geo-map
sampled_geohash_data2["neighborhood"].astype(str)
heatmap_scale = list()
threshold = [10,20,50,70,85,100]
for i in threshold :
    heatmap_scale.append(int(sampled_geohash_data2['count'].max() * (i/100.0)))

map_sampled_geohash_data2 = folium.Map(location=[40.730610, -73.935242], zoom_start=10)
map_sampled_geohash_data2.choropleth(geo_data=hoods, data=sampled_geohash_data2, \
                data_out = 'nyc_zip_test.json',
             columns=['neighborhood', 'count'],
             #threshold_scale= heatmap_scale,
             key_on='feature.properties.neighborhood',
             fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.9,nan_fill_color='white',
             legend_name='Number of Pickups')

/usr/local/lib/python3.10/dist-packages/folium/folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [86]:
map_sampled_geohash_data2

#MAPE

In [95]:
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [ ]:
# to numpy array - groupby geohash instead
count_scenario2 = sampled_data_grouped['geohash'].to_numpy()
count_scenario2

In [ ]:
MAPE(count_original,count_scenario2)

In [ ]:
MAPE(average_original,average_scenario2)

#RMSE

In [ ]:
#Defining RMSE function
def RMSE(Y_actual,Y_Predicted):
    rmse = np.sqrt(1/np.count_nonzero(Y_actual) * np.sum(np.power(Y_actual - Y_Predicted,2)))
    return rmse

In [ ]:
#Defining RMSE function
def RMSE(Y_actual,Y_Predicted):
    rmse = np.sqrt(1/np.count_nonzero(Y_actual) * np.sum(np.power(Y_actual - Y_Predicted,2)))
    return rmse

# Spearman correlation coefficient